In [1]:
import re
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from PIL import Image
import emoji 
from collections import Counter 
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


In [68]:
def date_time(s):
    pattern = '^([0-9]+(\/)([0-9]+)(\/)[0-9]+, ([0-9]+):([0-9]+)\s(PM|AM|am|pm) - )'
    result = re.match(pattern, s)
    if result:
        return True 
    return False

def contact(s):
    s=s.split(":")
    if len(s) == 2:
        return True 
    return False

def getmsg(line):
    splitline = line.split(' - ')
    date, time = splitline[0].split(', ')
    msg = " ".join(splitline[1:])
   
    if contact(msg):
        split_msg = msg.split(': ')
        author = split_msg[0]
        msg = " ".join(split_msg[1:])
    else:
        author = None
    return date, time, author, msg


<>:2: SyntaxWarning: invalid escape sequence '\/'
<>:2: SyntaxWarning: invalid escape sequence '\/'
/var/folders/rq/jmjrwgc95qd70xv7tvx6s7nm0000gp/T/ipykernel_3645/1766381712.py:2: SyntaxWarning: invalid escape sequence '\/'
  pattern = '^([0-9]+(\/)([0-9]+)(\/)[0-9]+, ([0-9]+):([0-9]+)\s(PM|AM|am|pm) - )'


In [69]:
data = []
conversation = 'WhatsApp Chat with Vishal Sir.txt'
with open(conversation, encoding="utf-8") as fp:
    fp.readline()
    msgBuffer = []
    date, time, author=None, None, None
    while True:
        line = fp.readline()
        if not line:
            break
        line = line.strip()

        if date_time(line):
            if len(msgBuffer) > 0 :
                data.append([date, time, author, " ".join(msgBuffer)])
            msgBuffer.clear()
            date, time, author, msg = getmsg(line)
            msgBuffer.append(msg)
        else:
            msgBuffer.append(line)

['3/31/21, 7:44\u202fPM', 'Sai Nithin Kadarla: Sir this too']
Sai Nithin Kadarla: Sir this too
['Sai Nithin Kadarla', 'Sir this too']
['3/31/21, 7:47\u202fPM', 'Sai Nithin Kadarla: <Media omitted>']
Sai Nithin Kadarla: <Media omitted>
['Sai Nithin Kadarla', '<Media omitted>']
['4/7/21, 5:59\u202fPM', 'Sai Nithin Kadarla: You deleted this message']
Sai Nithin Kadarla: You deleted this message
['Sai Nithin Kadarla', 'You deleted this message']
['4/7/21, 5:59\u202fPM', 'Sai Nithin Kadarla: <Media omitted>']
Sai Nithin Kadarla: <Media omitted>
['Sai Nithin Kadarla', '<Media omitted>']
['4/7/21, 5:59\u202fPM', 'Sai Nithin Kadarla: sir i did it']
Sai Nithin Kadarla: sir i did it
['Sai Nithin Kadarla', 'sir i did it']
['5/5/21, 9:37\u202fPM', 'Sai Nithin Kadarla: Sir , is online classes going on ?']
Sai Nithin Kadarla: Sir , is online classes going on ?
['Sai Nithin Kadarla', 'Sir , is online classes going on ?']
['6/30/21, 2:35\u202fPM', 'Sai Nithin Kadarla: Thank you sir']
Sai Nithin Kadarl

In [70]:
msgBuffer

['It’s ok sir', 'Any time']

In [71]:
df = pd.DataFrame(data, columns=["Date", "Time", "Contact", "Message"])
df['Date'] = pd.to_datetime(df['Date'])
data = df.dropna()

from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentiments = SentimentIntensityAnalyzer()
data['Positive'] = [sentiments.polarity_scores(i)['pos'] for i in data["Message"]]
data['Negative'] = [sentiments.polarity_scores(i)['neg'] for i in data["Message"]]
data['Neutral'] = [sentiments.polarity_scores(i)['neu'] for i in data["Message"]]

/var/folders/rq/jmjrwgc95qd70xv7tvx6s7nm0000gp/T/ipykernel_3645/1397526674.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'])
/var/folders/rq/jmjrwgc95qd70xv7tvx6s7nm0000gp/T/ipykernel_3645/1397526674.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Positive'] = [sentiments.polarity_scores(i)['pos'] for i in data["Message"]]
/var/folders/rq/jmjrwgc95qd70xv7tvx6s7nm0000gp/T/ipykernel_3645/1397526674.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [72]:
data.head(50)

,Date,Time,Contact,Message,Positive,Negative,Neutral
0,2021-03-31,7:44 PM,Sai Nithin Kadarla,Sir this too,0.000,0.0,1.000
1,2021-03-31,7:47 PM,Sai Nithin Kadarla,<Media omitted>,0.000,0.0,1.000
2,2021-04-07,5:59 PM,Sai Nithin Kadarla,You deleted this message,0.000,0.0,1.000
3,2021-04-07,5:59 PM,Sai Nithin Kadarla,<Media omitted>,0.000,0.0,1.000
4,2021-04-07,5:59 PM,Sai Nithin Kadarla,sir i did it,0.000,0.0,1.000
5,2021-05-05,9:37 PM,Sai Nithin Kadarla,"Sir , is online classes going on ?",0.000,0.0,1.000
6,2021-06-30,2:35 PM,Sai Nithin Kadarla,Thank you sir,0.556,0.0,0.444
8,2021-12-03,4:20 PM,Vishal Sir,<Media omitted>,0.000,0.0,1.000
9,2021-12-03,4:21 PM,Vishal Sir,<Media omitted>,0.000,0.0,1.000
10,2021-12-03,4:21 PM,Vishal Sir,<Media omitted>,0.000,0.0,1.000


In [82]:
pos = sum(data['Positive'])
neg = sum(data['Negative'])
neu = sum(data['Neutral'])

def score(a,b,c):
    if a>b and a>c:
        print('postive')
    elif b>a and b>c:
        print('negative')
    else:
        print("neutral")

score(pos, neg, neu), pos, neg, neu

neutral


(None, 23.376, 0.41700000000000004, 99.207)

In [83]:
pos

23.376